In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score

In [32]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')
X_train, y_train = train_data.drop('class of diagnosis', axis=1), train_data['class of diagnosis']
X_test, y_test = test_data.drop('class of diagnosis', axis=1), test_data['class of diagnosis']


train_data.head()
test_data['class of diagnosis']
n_features = X_train.shape[1]

In [ ]:
model = RandomForestClassifier(n_estimators= 100, random_state=123, n_jobs=-1)
param_grid1 = {
    'max_depth': [3, 10, None], 'max_features': ['sqrt', n_features], 'min_samples_split': [2,5,10],
    'n_estimators': [10, 100, 500]
}

grid_search1 = GridSearchCV(model, param_grid=param_grid1)
grid_search1.fit(X_train, y_train)
model1 = grid_search1.best_estimator_
pd.DataFrame(grid_search1.cv_results_)



In [ ]:
param_grid2 = {
    'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 5, 10, 20]
}
grid_search2 = GridSearchCV(model, param_grid=param_grid2)
grid_search2.fit(X_train, y_train)
model2 = grid_search2.best_estimator_
pd.DataFrame(grid_search2.cv_results_)




In [ ]:
param_grid3 = {
    'min_samples_split': [4,5,6] , 'min_samples_leaf': [1,2,3]
}
grid_search3 = GridSearchCV(model, param_grid=param_grid2)
grid_search3.fit(X_train, y_train)
model3 = grid_search3.best_estimator_
pd.DataFrame(grid_search3.cv_results_)


In [ ]:
final = model3
final.fit(X_train, y_train)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, final.predict_proba(X_test).T[1])
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b')
plt.plot([0,1],[0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()